# Parallel computing with dask

<div class="alert alert-info">
<i class="fa-question-circle fa" style="font-size: 22px;color:#666;"></i> Overview
    <br>
    <br>
    <b>Questions</b>
    <ul>
        <li>Dask</li>
    </ul>
    <b>Objectives</b>
    <ul>
        <li>Learn about hvplot</li>
    </ul>
</div>